In [33]:
!pip install langchain
!pip install langchain-community
!pip install datasets
# !pip install -U datasets
!pip install unstructured
!pip install faiss-cpu
!pip install peft
!pip install evaluate[metrics]
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached matplotlib-3.10.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
Using cached matplotlib-3.10.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached contourpy-1.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (326 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.58.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
Using cached kiwisolver-1.4.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)


In [9]:
#!/usr/bin/env python3
"""Evaluate Qwen2.5-1.5B *before* and *after* LoRA-SFT on the Russian XQuAD benchmark.

Benchmark (default 300 samples):
  • **XQuAD (ru)** – exact_match, f1 (SQuAD v1 metrics) + BERTScore.

Prompt template: using `tokenizer.apply_chat_template(..., add_generation_prompt=True)` as in production.

Usage:
    python eval_pre_post_sft_xquad.py \
        --lora ./sft_checkpoints_ru_lora/ \
        --sample 300       # optional: number of examples to use
"""
import argparse
from argparse import Namespace
import json
import os
from pathlib import Path
import torch
from tqdm import tqdm
from datasets import load_dataset
from evaluate import load as load_metric
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel
from datetime import datetime

BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------------------------------------------------------
# Single tokenizer instance for speed
# -----------------------------------------------------------------------------
print("Loading tokenizer...")
TOKENIZER = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

# -----------------------------------------------------------------------------
# Pipelines
# -----------------------------------------------------------------------------
def build_pipeline(model):
    return pipeline(
        "text-generation",
        model=model,
        tokenizer=TOKENIZER,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.8,
        top_k=20,
        device=DEVICE if not hasattr(model, "hf_device_map") else None,
    )


def load_base_pipeline():
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL, trust_remote_code=True, device_map="auto"
    )
    return build_pipeline(model)


def load_lora_pipeline(lora_dir: str):
    base = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL, trust_remote_code=True, device_map="auto"
    )
    model = PeftModel.from_pretrained(base, lora_dir, trust_remote_code=True)
    model = model.merge_and_unload()
    return build_pipeline(model)

# -----------------------------------------------------------------------------
# Prompt helpers
# -----------------------------------------------------------------------------

def _format_messages(context: str, question: str):
    return [
        {"role": "system", "content": "Ты — полезный AI‑ассистент. Отвечай на вопросы, используя предоставленный контекст."},
        {"role": "user", "content": f"Контекст:\n{context}\n\nВопрос: {question}"},
    ]


def qa_prompt(context: str, question: str) -> str:
    return TOKENIZER.apply_chat_template(
        _format_messages(context, question),
        tokenize=False,
        add_generation_prompt=True,
    )

# -----------------------------------------------------------------------------
# Generation helper
# -----------------------------------------------------------------------------

def generate_answer(pipe, prompt: str) -> str:
    full = pipe(prompt)[0]["generated_text"]
    return full[len(prompt):].strip() if full.startswith(prompt) else full.strip()

# -----------------------------------------------------------------------------
# XQuAD evaluation
# -----------------------------------------------------------------------------

def eval_xquad(pipe, n_samples: int, model_name: str = "base_model"):
    # Load Russian XQuAD (SQuAD v1 style)
    ds = load_dataset("google/xquad", "xquad.ru", split="validation")
    if n_samples and n_samples < len(ds):
        ds = ds.shuffle(seed=42).select(range(n_samples))

    # Metrics
    squad_metric = load_metric("squad")
    bertscore_metric = load_metric("bertscore")

    preds = []
    refs = []
    bs_preds = []
    bs_refs = []
    
    # Prepare logging
    log_dir = Path("eval_logs")
    log_dir.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = log_dir / f"xquad_eval_{model_name}_{timestamp}.json"
    log_data = {
        "model": model_name,
        "timestamp": timestamp,
        "samples": [],
        "metrics": {}
    }

    for sample in tqdm(ds, desc="XQuAD_ru"):
        context = sample["context"]
        question = sample["question"]
        gold_texts = sample["answers"]["text"]
        gold_starts = sample["answers"]["answer_start"]

        prompt = qa_prompt(context, question)
        pred = generate_answer(pipe, prompt)

        # Log sample data
        sample_log = {
            "id": sample["id"],
            "context": context,
            "question": question,
            "gold_answers": gold_texts,
            "model_answer": pred,
            "prompt": prompt
        }
        log_data["samples"].append(sample_log)

        # SQuAD format
        preds.append({"id": sample["id"], "prediction_text": pred})
        refs.append({"id": sample["id"], "answers": {"text": gold_texts, "answer_start": gold_starts}})

        # BERTScore expects single ref; take first if multiple
        bs_preds.append(pred)
        bs_refs.append(gold_texts[0] if gold_texts else "")

    # Compute metrics
    squad_res = squad_metric.compute(predictions=preds, references=refs)
    bert_res = bertscore_metric.compute(
        predictions=bs_preds,
        references=bs_refs,
        lang="ru"
    )

    metrics = {
        "exact_match": squad_res["exact_match"],
        "f1": squad_res["f1"],
        "bertscore_precision": sum(bert_res["precision"]) / len(bert_res["precision"]),
        "bertscore_recall": sum(bert_res["recall"]) / len(bert_res["recall"]),
        "bertscore_f1": sum(bert_res["f1"]) / len(bert_res["f1"]),
    }
    
    # Add metrics to log data
    log_data["metrics"] = metrics
    
    # Save logs to file
    with open(log_file, "w", encoding="utf-8") as f:
        json.dump(log_data, f, ensure_ascii=False, indent=2)

    return metrics

Loading tokenizer...


In [10]:
# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "--lora", required=True, help="Path to LoRA checkpoint (sft_output_dir)"
# )
# parser.add_argument(
#     "--sample", type=int, default=300,
#     help="Number of examples from XQuAD to evaluate"
# )
# args = parser.parse_args()

args = Namespace(
    lora="/workspace-SR008.fs2/zhogov/test/sft_checkpoints_ru_lora/",
    sample=880
)

print("\n=== Base model ===")
base_pipe = load_base_pipeline()
base_metrics = eval_xquad(base_pipe, args.sample, "base_model_base")
print("XQuAD (ru):", base_metrics)

print("\n=== After LoRA-SFT ===")
lora_pipe = load_lora_pipeline(args.lora)
lora_metrics = eval_xquad(lora_pipe, args.sample, "lora_sft_base")
print("XQuAD (ru):", lora_metrics)


=== Base model ===


Device set to use cuda:0
XQuAD_ru: 100%|██████████| 880/880 [13:09<00:00,  1.11it/s]


XQuAD (ru): {'exact_match': 0.3409090909090909, 'f1': 18.879509115081774, 'bertscore_precision': 0.6107046019963243, 'bertscore_recall': 0.7611905857243321, 'bertscore_f1': 0.6764862799170342}

=== After LoRA-SFT ===


Device set to use cuda:0
XQuAD_ru: 100%|██████████| 880/880 [02:12<00:00,  6.63it/s]


XQuAD (ru): {'exact_match': 30.0, 'f1': 49.2187858563608, 'bertscore_precision': 0.7236526182768019, 'bertscore_recall': 0.7382793841036883, 'bertscore_f1': 0.7299502228471366}
